In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
import torch
import time,copy
import math
from sklearn.metrics import accuracy_score,f1_score
from tqdm import tqdm

ModuleNotFoundError: No module named 'pandas'

#### encoding 4 bits into one hot 

In [ ]:
def encod_map(x):
    e = np.zeros(16).astype(int)
    index = int("".join(str(y) for y in x), 2)
    e[index] = 1
    return  e

In [ ]:
def one_hot_vec(X):

    new_X = np.zeros((X.shape[0],2**(X.shape[1])))
    for i in range(X.shape[0]):
        new_X[i] = encod_map(X[i])
    # print(new_X[:2])
    # print(np.where(np.sum(new_X,axis=1) != 1))
    # print(np.where(np.sum(x_new_val,axis=1) != 1)
    return new_X

In [ ]:
X = np.random.randint(1,10,size=(10000,4))%2
X = one_hot_vec(X)
X.shape

(10000, 16)

In [ ]:
df = pd.DataFrame(X)
df.head()
df.to_csv('./input_data.csv',index = False,header = False)

#### batch_size

In [ ]:
train_batch_sz = 16
val_batch_sz = 64
varience = 0

#### train_test_split

In [ ]:
load_df = pd.read_csv("./input_data.csv",header=None)
load_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
load_df['max'] = load_df.loc[:,:-1:-1].idxmax(axis = "columns")
load_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,max
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,14
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11


In [ ]:
train_df,val_df = train_test_split(load_df,test_size=0.2,random_state=4,stratify=load_df['max'])

In [ ]:
print(train_df.shape,val_df.shape)

(8000, 17) (2000, 17)


In [ ]:
class GenerateDataset(Dataset):
  def __init__(self, data_frame,subtask):
    self.data_frame = data_frame
    self.subtask = subtask 

  def __len__(self):
    return len(self.data_frame)
    
  def __getitem__(self, idx):
      
    if self.subtask != "test":
      label = self.data_frame.iloc[idx, -1]
    else:
      return torch.tensor(self.data_frame.iloc[idx])

    return (torch.tensor(self.data_frame.iloc[idx,:-1],dtype = float), torch.tensor(label))

In [ ]:
train_set = GenerateDataset(train_df,'train')
val_set = GenerateDataset(val_df,'val')

In [ ]:
for a,b in train_set:
    print(a,b)
    break
for a,b in val_set:
    print(a,b)
    break

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       dtype=torch.float64) tensor(10)
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64) tensor(6)


In [ ]:
train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,max
9365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,10
1127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,14
588,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
9490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15
1822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,10
3709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,14
8549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,14
579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13


In [ ]:
train_loader = DataLoader(train_set,batch_size = train_batch_sz,shuffle = True)
val_loader = DataLoader(val_set,batch_size = val_batch_sz,shuffle = False)

#### backprop model

In [ ]:
class BaselineBackpropModel(nn.Module):
    def __init__(self, in_sz=16, out_sz=16, layers=[16, 7]):
        super().__init__()
        self.relu = nn.ReLU()

        self.encoder = nn.Sequential(
            nn.Linear(in_sz, layers[0]),
            self.relu,
            nn.Linear(layers[0], layers[1]),
        )

        self.decoder = nn.Sequential(
            nn.Linear(layers[1], layers[0]),
            self.relu,
            nn.Linear(layers[0], out_sz),
            nn.LogSoftmax(dim=1)
        )

    def encode_data(self, X):
        return self.encoder(X)

    def decode_data(self, X):
        return self.decoder(X)

    def forward(self, X):
        encoded_sig = self.encode_data(X)
        normalized_sig = self.normalize(encoded_sig)
        corr_sig = self.add_noise(normalized_sig)
        decoded_sig = self.decode_data(corr_sig)
        return decoded_sig

    def normalize(self, sig):
        normalizer = torch.zeros(sig.size()[0], 1)
        normalizer[:, 0] = torch.linalg.norm(sig, dim=1)
        normalizer = torch.tile(normalizer,(1,sig.size()[1]))
        sig = torch.div(sig, normalizer)
        sig = sig * 7
        return sig

    def add_noise(self, msg):
        msg = msg + (varience ** 0.5) * torch.randn(*msg.size())
        return msg


In [ ]:
# y = torch.tensor([[1], [3]],dtype=float)
# print(y.size())
# # y = y.reshape(y.size()[0],1)
# c = torch.tile(y, (1,2))
# print(c)
# b = torch.tensor([[10,10],[20,20]],dtype=float)
# print(torch.div(b,c))

torch.Size([2, 1])
tensor([[1., 1.],
        [3., 3.]], dtype=torch.float64)
tensor([[10.0000, 10.0000],
        [ 6.6667,  6.6667]], dtype=torch.float64)


In [ ]:
# torch.linalg.norm(torch.tensor([1,2,3],dtype=float))

tensor(3.7417, dtype=torch.float64)

In [ ]:
# a = torch.tensor([[1,2,3],[4,5,6]],dtype=float)
# normalizer = torch.zeros(a.size()[0], 1)
# print(torch.linalg.norm(a, dim=1))
# print(torch.linalg.norm(a, dim=0))
# print(torch.linalg.norm(a))

tensor([3.7417, 8.7750], dtype=torch.float64)
tensor([4.1231, 5.3852, 6.7082], dtype=torch.float64)
tensor(9.5394, dtype=torch.float64)


In [ ]:
backprop = BaselineBackpropModel()
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(backprop.parameters(), lr=0.01)

In [ ]:
import time
start_time = time.time()

epochs = 10

fp = open('backprop_train.txt', "w")
fp1 = open('backprop_val.txt','w')
for i in tqdm(range(epochs),desc = "Progress Bar"):

    trn_corr = 0
    tst_corr = 0

    for b, data in enumerate(train_loader):

        b += 1
        btch_corr = 0
        inputs, labels = data
        inputs = inputs.float()
        optimizer.zero_grad()
        y_pred = backprop(inputs)
        
        loss = criterion(y_pred,labels)

        y_max = torch.argmax(y_pred,dim = 1)
        btch_corr = torch.sum(y_max == labels)
        trn_corr += btch_corr

        loss.backward()
        optimizer.step()

        if b % 20 == 19:
            fp.write(
                f'epoch: {i}  train loss: {loss.item():10.8f}  train batch accuracy: {btch_corr.item()*100/(train_batch_sz):7.3f}%')
            fp.write('\n')

    with torch.no_grad():

        for c,test in enumerate(val_loader):

            c += 1

            val_inputs,val_labels = test
            val_inputs = val_inputs.float()
            val_pred = backprop(val_inputs)
            loss = criterion(val_pred,val_labels)

            val_max = torch.argmax(val_pred,dim = 1)
            tst_corr += torch.sum(val_max == val_labels)

            fp1.write(
                f'epoch: {i}  val loss: {loss.item():10.8f}  val accuracy: {tst_corr.item()*100/(val_batch_sz*c):7.3f}%')
            fp1.write('\n')

fp.close()
fp1.close()

print(f'\nDuration: {time.time() - start_time:.0f} seconds')


Progress Bar:   0%|          | 0/10 [00:00<?, ?it/s]/home/lohith/chida/environments/basic/lib/python3.7/site-packages/torch/autograd/__init__.py:175: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass
Progress Bar: 100%|██████████| 10/10 [01:02<00:00,  6.29s/it]


Duration: 63 seconds


In [ ]:
# Evaluation mode
backprop.eval()
all_labels = []
all_predicted = []
with torch.no_grad():
    for data in val_loader:
        inputs, labels = data
        inputs = inputs.float()
        outputs = backprop(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.to('cpu')
        all_labels.append(labels)
        all_predicted.append(predicted)
    a1,a2 = all_labels,all_predicted
    a1 = torch.cat(a1)
    a2 = torch.cat(a2)
    print(f'F1 score is {np.mean(f1_score(a1,a2,average = None))}') # Printing F1 score
    print(f'Accuracy score is {np.mean(accuracy_score(a1,a2))}') # Printing accuracy_score


F1 score is 1.0
Accuracy score is 1.0


#### SPSA model

In [ ]:
class SpsaModel(BaselineBackpropModel):

    def __init__(self, in_sz=16, out_sz=16, layers=[16, 7]):
        super().__init__(in_sz, out_sz, layers)
        self.criterion = nn.NLLLoss()

    def stop_backprop(self):
        for param in self.parameters():
            param.requires_grad = False
        return

    def calculate_loss(self, inputs, lables):
        y_pred = self.forward(inputs)
        loss = self.criterion(y_pred, labels)
        return loss.item()

    def store_wb(self):
        theta = []
        delta = []
        for param in self.parameters():
            inter = torch.randn(*param.size())
            inter[inter >= 0] = 1
            inter[inter < 0] = -1

            if param.dim() == 1:
                for i in param:
                    theta.append(i)
                for j in inter:
                    delta.append(j)

            elif param.dim() == 2:
                temp = torch.flatten(param)
                for a in temp:
                    theta.append(a)
                
                intrmd = torch.flatten(inter)
                for b in intrmd:
                    delta.append(b)
                # for element in inter:
                #     for i in element:
                #         delta.append(i)
            else:
                raise ValueError("dim should be 1 or 2")
        return torch.from_numpy(np.array(theta)).float(), torch.from_numpy(np.array(delta)).float()

    def update_parameters(self, theta, temp, factor):
        theta.data.add_(factor*temp)
        pos = 0
        for names,param in self.named_parameters():
            if "bias" in names:
                var = int(param.size()[0])
                param.data.copy_(theta[pos:(pos+var)].data)
            elif "weight" in names:
                var = int(param.size()[0] * param.size()[1])
                temp = torch.reshape(theta[pos:(pos+var)], param.size())
                param.data.copy_(temp.data)
            else:
                raise ValueError("dim should be 1 or 2")
            pos += var
        return

    def updt_spsa(self, inputs, labels, a, A, c, alpha, gamma, num_iter):

        for k in range(1, num_iter):
            theta, delta = self.store_wb()
            saviour = theta.detach().clone()
            lr = (a/((k+A)**alpha))
            curr_c = (c/(k**gamma))
            temp = curr_c * delta
            self.update_parameters(theta, temp, 1)
            grad = self.calculate_loss(inputs, labels)
            self.update_parameters(theta, temp, -2)
            grad -= self.calculate_loss(inputs, labels)
            grad = grad/(2 * temp)
            self.update_parameters(saviour, grad, (-1 * lr))
        return


In [ ]:
# for names,param in spsa.named_parameters():
#     print(names,param.dim(),*param.size())

In [ ]:
spsa = SpsaModel()
print(spsa)
spsa.stop_backprop()

SpsaModel(
  (relu): ReLU()
  (encoder): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=7, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=7, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): LogSoftmax(dim=1)
  )
  (criterion): NLLLoss()
)


In [ ]:
torch.autograd.set_detect_anomaly(True)
start_time = time.time()
epochs = 1
num_iter = 30
fp = open('spsa_train.txt', "w")
fp1 = open('spsa_val.txt','w')
for i in range(epochs):

    spsa_trn_corr = 0
    spsa_tst_corr = 0

    for b, data in tqdm(enumerate(train_loader),desc = "batch progress"):
        b += 1
        spsa_btch_corr = 0
        inputs, labels = data
        inputs = inputs.float()
        y_pred = spsa.forward(inputs)
        optimizer.zero_grad()
        
        loss = criterion(y_pred, labels)
        y_max = torch.argmax(y_pred, dim=1)
        spsa_btch_corr = torch.sum(y_max == labels)
        spsa_trn_corr += spsa_btch_corr

        spsa.updt_spsa(inputs, labels, 0.05, 25, 0.1, 0.9, 0.3, num_iter)

        optimizer.step()

        if b % 25 == 24:
            fp.write(
                f'epoch: {i} train loss: {loss.item():10.8f} train batch accuracy: {spsa_btch_corr.item()*100/(train_batch_sz):7.3f}%')
            fp.write('\n')

    for c, test in enumerate(val_loader):

        c += 1

        val_inputs, val_labels = test
        val_inputs = val_inputs.float()
        val_pred = spsa.forward(val_inputs)
        loss = criterion(val_pred, val_labels)

        val_max = torch.argmax(val_pred, dim=1)
        spsa_tst_corr += torch.sum(val_max == val_labels)

        fp1.write(
            f'epoch: {i}  val loss: {loss.item():10.8f}  val accuracy: {spsa_tst_corr.item()*100/(val_batch_sz*c):7.3f}%')
        fp1.write('\n')

fp.close()
fp1.close()

batch progress: 500it [02:23,  3.48it/s][00:00<?, ?it/s]
spsa progress bar: 100%|██████████| 1/1 [02:24<00:00, 144.32s/it]
